In [1]:
# WRITTEN BY CHRISTOPHER STONELL 2017
# GENERATES ACHS INDICATORS AND CONTROL CHARTS FOR PACU INDICATORS
# PLEASE ACKNOWEDGE THE SOURCE IF YOU CHOOSE TO USE THIS CODE.

# clear workspace
rm(list=ls())

# Load required libraries
library("gridExtra")
library("devtools")
library("qcc")
library("formattable")
library("htmltools")
library("webshot")
library("plyr")
library("knitr")
library(ggplot2)
library(digest) # for anonymise.R function to hash MRN

# Set directories
anaes.data.directory <- "~/Downloads/Anaesthetic"
#functions.directory  <- "~/MEGAsync/QA\ Data/R Functions"
pacu.data.directory  <- "~/Downloads/Data"
output.directory     <- "~/Downloads/Output"

# Load required functions and routines
source_url("https://raw.githubusercontent.com/vxoli/R/master/Load%20%26%20Combine.R")
source_url("https://raw.githubusercontent.com/vxoli/R/master/export_formattable.R")
source_url("https://raw.githubusercontent.com/vxoli/R/master/anonymise.R")
# source("~/MEGAsync/QA Data/R Functions/anonymise.R")
#setwd(functions.directory)
#source("Load\ &\ Combine.R")

# Retrieve MOT Data
setwd(anaes.data.directory)
# **consider amend function so pass path and fxn reads files then returns to default path**
# ** also consider using zip files to store the data - perhaps one for mot and one for pac for each time period - then wouldn't need folder per data set but zip file per data set
mot.data <- data.frame(CombineAll())

# Drop Patient Name fields from data.frame
mot.data <- mot.data[, -which(names(mot.data) %in% c("Patient.First.Name", "Patient.Last.Name"))]
# Anonymise MRN with hash function
cols_to_anon <- c("MRN")
mot.data[,cols_to_anon] <- anonymise(mot.data,cols_to_anon)

# Modify dates from excel to work in R
# **could this be written as a fxn to be called with the format to convert and format to return**
mot.data$date = as.character(mot.data$Operation.Date)
mot.data$date = as.Date(mot.data$Operation.Date,format = "%d-%b-%Y")
mot.data$Month_Yr <- format(as.Date(mot.data$date), "%Y-%m")

# Clean up names in data
# Correct some duplicate names and remove suffex - Anaes Consultant from some
levels(mot.data$Anaes.1.Name) <- c(levels(mot.data$Anaes.1.Name), "COLLARD,Caroline", "SLYKERMAN,Julia", "HUANG,Dennis", "SINGH,Raman")
mot.data$Anaes.1.Name[mot.data$Anaes.1.Name == "COLLARD,Caroline - Anaes Cons"] <- "COLLARD,Caroline"
mot.data$Anaes.1.Name[mot.data$Anaes.1.Name == "SLYKERMAN,Julia - Anaes Consultant"] <- "SLYKERMAN,Julia"
mot.data$Anaes.1.Name[mot.data$Anaes.1.Name == "HUANG,Dennis - Anaes Consultant"] <- "HUANG,Dennis"
mot.data$Anaes.1.Name[mot.data$Anaes.1.Name == "SINGH,Raman - Anaes Consultant"] <- "SINGH,Raman"

# Populate PACU.ICU.WARD column from discharge column
#mot.data$PACU.ICU.WARD[mot.data$Ward.To.Code[0:2] == "ICU"] <- "ICU"
mot.data$PACU.ICU.WARD[substring(mot.data$Ward.To.Code,1,3) == "ICU"] <- "ICU"
mot.data$PACU.ICU.WARD[substring(mot.data$Ward.To.Code,1,3) != "ICU"] <- "PACU"

monthlycases <- table(mot.data$Month_Yr, mot.data$PACU.ICU.WARD)[,c("ICU","PACU")]


Loading required package: usethis
Package 'qcc' version 2.7
Type 'citation("qcc")' for citing this R package in publications.
Registered S3 methods overwritten by 'tibble':
  method     from  
  format.tbl pillar
  print.tbl  pillar
ℹ SHA-1 hash of file is 6f5e9bff288027319a0cdeb7b0ca08a10f5042d7
ℹ SHA-1 hash of file is 1db7e441c3d243f0d8ac49ed8e2da02e4f038dc2
ℹ SHA-1 hash of file is 014ca25bc3f8cc695b1a6e679fe8078e3d5b70e6


In [2]:
## Plot and save stacked barplot of MOT monthly activity seperated into ICU and PACU 
# **consider using a function called with filename properties to open the file and close after plot**
setwd(output.directory)
filename.prefix <- paste(min(mot.data$Month_Yr), "-", max(mot.data$Month_Yr)," ")
filename <- paste(filename.prefix, "MOT-throughput-barplot.pdf")
pdf(filename, height = 7, width =12 )
barplot(t(monthlycases), 
        ylab="# cases/ month", 
        las=2, 
        main=paste("Cases through MOT by month\nSeperated by discharge to ICU or PACU\nFor the period ",min(mot.data$Month_Yr)," to ",max(mot.data$Month_Yr)), 
        legend.text=c("ICU","PACU"),
        args.legend = list(
          x="topright",
          inset=c(-0.04,0),
          bty = "n"
        )
        )
dev.off()


png 
  2

In [108]:
# Start analysis of PACU data
setwd(pacu.data.directory)
pacu.data <- data.frame(CombineAll())
# Drop patient names from pacu.data
pacu.data <- pacu.data[, -which(names(pacu.data) %in% c("Patient.First.Name", "Patient.Last.Name"))]
# Anonymise MRN with hash function
cols_to_anon <- c("MRN")
pacu.data[,cols_to_anon] <- anonymise(pacu.data,cols_to_anon)

# Normalise dates & accomodate the timestamp field
# pacu.data$date <- as.character(pacu.data$In.Recovery.At) - do i need this line???
pacu.data$date <- as.Date(pacu.data$In.Recovery.At,format = "%d/%m/%Y  %I:%M:%S %p")
pacu.data$Month_Yr <- format(as.Date(pacu.data$date), "%Y-%m")
pacu.data$Day_Month_Yr <- format(as.Date(pacu.data$date), "%Y-%m-%d")
pacu.data$Answer <- as.character(pacu.data$Answer)

# Adverse events in ORMIS are:
events.descriptors <- c("ANAES RESP INTERVENT", "ANAES RESP INTERVENTION", "RESP COMPLICATION", "BLOOD FLUID LOSS", "CARDIO / RESP ARREST", "HAEMODYNAMIC COMP", "HYPOTHERMIA <36 DEG", "OTHER", "PERSISTENT PONV", "PROLONGED STAY > 2 HRS", "PROLONGED UNCONSC", "REACTION", "REINTUB/ VENTILATION", "RESP COMPLICATION", "RETURN TO OR", "UNPLANNED ADMISSION ICU", "ANTIEMETICS", "ANAESTH R/V- PAIN", "PAIN R/V ANAES CONS")

# Use PACU cases from MOT data for pacu total cases per month
events.adverse <- data.frame(matrix(NA, nrow=length(unique(mot.data$Month_Yr)), ncol=length(events.descriptors)+2))
# Data frame had date as 1st col, then total PACU cases per month as 1 column and event totals as other columns.
names(events.adverse) <- c("date","total.pacu.cases",events.descriptors)
# add dates from mot.data to dates column. 
events.adverse$date <- sort(unique(pacu.data$Month_Yr), decreasing=FALSE)
# add total cases per month through PACU to column
events.adverse$total.pacu.cases <- monthlycases[,"PACU"]

##REMOVE FROM HERE DOWN
#First do frequency table by month
# enter event counts to each column - ensure dates match up
for (i in events.descriptors){
  t <- as.data.frame(table(pacu.data$Month_Yr, pacu.data$Answer == i))
  t <- subset(t, t["Var2"]==TRUE)
  events.adverse[,i] <- t["Freq"]
} # End for i loop
# Alternate method to count by date and event - may be quicker/ better - seems to not work correctly ??syntax
# events.adverse[,i] <- with(pacu.data, tapply(pacu.data$Answer==i, pacu.data$Month_Yr, FUN=function(x) length(unique(x))))
# If no adverse events in a category the column will remain filled with NA - convert these to zero for calculations
events.adverse[is.na(events.adverse)] <- 0

events.adverse$"Resp V Serious" <- events.adverse$"CARDIO / RESP ARREST" + events.adverse$"REINTUB/ VENTILATION"
events.descriptors <- c(events.descriptors, "Resp Event", "Resp Serious", "Resp V Serious")
events.adverse$"Resp Event" <- events.adverse$"ANAES RESP INTERVENT" + events.adverse$"CARDIO / RESP ARREST" + events.adverse$"REINTUB/ VENTILATION" + events.adverse$"RESP COMPLICATION"
events.adverse$"Resp Serious" <- events.adverse$"ANAES RESP INTERVENT" + events.adverse$"ANAES RESP INTERVENTION" + events.adverse$"CARDIO / RESP ARREST" + events.adverse$"REINTUB/ VENTILATION"

names(events.adverse)[names(events.adverse)=="CARDIO / RESP ARREST"] <- "CARDIO-RESP ARREST"
events.descriptors <- names(events.adverse[2:length(events.adverse)])
pacu.data$Answer[grep("CARDIO / RESP ARREST", pacu.data$Answer, ignore.case=TRUE)] <- as.character("CARDIO-RESP ARREST")
names(events.adverse)[names(events.adverse)=="REINTUB/ VENTILATION"] <- "REINTUBATION"
events.descriptors <- names(events.adverse[2:length(events.adverse)])
pacu.data$Answer[grep("REINTUB/ VENTILATION", pacu.data$Answer, ignore.case=TRUE)] <- as.character("REINTUBATION")

events.infrequent.descriptors <- c(events.descriptors[4], events.descriptors[11], events.descriptors[13],events.descriptors[14]) #Cardiac Arrest, Reintubation, Return to OR, Unplanned ICU
events.to.skip <- c(events.descriptors[7], events.descriptors[9], events.descriptors[10], events.descriptors[19], events.infrequent.descriptors) # Other, Prolonged Unconc, Reaction, Resp V Serious


ERROR: Error in `$<-.data.frame`(`*tmp*`, "PERSISTENT PONV", value = c("PERSISTENT PONV", : replacement has 23 rows, data has 6


In [66]:
#First do frequency table by month
# enter event counts to each column - ensure dates match up
for (i in events.descriptors){
  t <- as.data.frame(table(pacu.data$Month_Yr, pacu.data$Answer == i))
  t <- subset(t, t["Var2"]==TRUE)
  events.adverse[,i] <- t["Freq"]
} # End for i loop
# Alternate method to count by date and event - may be quicker/ better - seems to not work correctly ??syntax
# events.adverse[,i] <- with(pacu.data, tapply(pacu.data$Answer==i, pacu.data$Month_Yr, FUN=function(x) length(unique(x))))
# If no adverse events in a category the column will remain filled with NA - convert these to zero for calculations
events.adverse[is.na(events.adverse)] <- 0

# Respiratory event = Anaes intervent + Cardio/ Resp Arrest + Reintub + Resp Complication
events.adverse$"Resp V Serious" <- events.adverse$"CARDIO / RESP ARREST" + events.adverse$"REINTUB/ VENTILATION"
events.descriptors <- c(events.descriptors, "Resp Event", "Resp Serious", "Resp V Serious")
events.adverse$"Resp Event" <- events.adverse$"ANAES RESP INTERVENT" + events.adverse$"CARDIO / RESP ARREST" + events.adverse$"REINTUB/ VENTILATION" + events.adverse$"RESP COMPLICATION"
# Serious Resp event = Anaes intervent + Cario Resp Arrest + Reintub
events.adverse$"Resp Serious" <- events.adverse$"ANAES RESP INTERVENT" + events.adverse$"ANAES RESP INTERVENTION" + events.adverse$"CARDIO / RESP ARREST" + events.adverse$"REINTUB/ VENTILATION"
# V Serous Resp Event = Arrest + Reintub

# Change CARDIO / RESP ARREST as this label causes an error later when used for file nameing
names(events.adverse)[names(events.adverse)=="CARDIO / RESP ARREST"] <- "CARDIO-RESP ARREST"
events.descriptors <- names(events.adverse[2:length(events.adverse)])
# Update pacu.data$Answer with new event name
pacu.data$Answer[grep("CARDIO / RESP ARREST", pacu.data$Answer, ignore.case=TRUE)] <- as.character("CARDIO-RESP ARREST")
# Change REINTUB/ VENTILATION as this label causes an error later when used for file nameing
names(events.adverse)[names(events.adverse)=="REINTUB/ VENTILATION"] <- "REINTUBATION"
events.descriptors <- names(events.adverse[2:length(events.adverse)])
# Update pacu.data$Answer with new event name
pacu.data$Answer[grep("REINTUB/ VENTILATION", pacu.data$Answer, ignore.case=TRUE)] <- as.character("REINTUBATION")

# Unplanned ICU admission should be calculated from the mot data detecting dispatity between
# planned discharge ward and actual discharge ward

# Remove unnecessary columns for charting etc or use new vector with only columns needed for charting
# Update the event descriptors in the vector
events.infrequent.descriptors <- c(events.descriptors[4], events.descriptors[11], events.descriptors[13],events.descriptors[14]) #Cardiac Arrest, Reintubation, Return to OR, Unplanned ICU
events.to.skip <- c(events.descriptors[7], events.descriptors[9], events.descriptors[10], events.descriptors[19], events.infrequent.descriptors) # Other, Prolonged Unconc, Reaction, Resp V Serious


ERROR: Error in `$<-.data.frame`(`*tmp*`, "Resp V Serious", value = integer(0)): replacement has 0 rows, data has 6


In [68]:
# Do plots for each event descriptor & write to pdf
events.to.chart <- events.descriptors[! events.descriptors %in% events.to.skip]
events.to.chart <- events.to.chart[2:length(events.to.chart)]
for (i in events.to.chart){
  setwd(output.directory)
  filename.prefix <- paste(min(mot.data$Month_Yr), "-", max(mot.data$Month_Yr)," ")
  filename <- gsub("/", "", paste(filename.prefix, i," p-chart.pdf"))
  pdf(filename, height = 7, width =12 )

    qcc(data = events.adverse[i],
      type="p",
      sizes=events.adverse$total.pacu.cases,
      nsigmas=3,
      labels=events.adverse$date,
      axes.las=2,
      data.name = i, #c(i, min(mot.data$Month_Yr), "-", max(mot.data$Month_Yr)),
      add.stats=FALSE,
      xlab= "Date",
      ylab = "Proportion",
      title = paste(i, "\n", min(mot.data$Month_Yr), " to ", max(mot.data$Month_Yr))
      ) # Close qcc
    dev.off()
  } # end for i


ERROR: Error in `[.data.frame`(events.adverse, i): undefined columns selected


In [6]:
# Use qcc g-chart for plot of days between events
# use for return to or, cardiac arrest, reintubation, unplanned admiss icu - might need to consider larger data set so longer time span analysed
# Plot g-charts
for (i in events.infrequent.descriptors) {
  if (i %in% (events.infrequent.descriptors[2|3])) next
 t <- as.data.frame(table(pacu.data$Day_Month_Yr, pacu.data$Answer == i))
 t <- subset(t, t["Var2"]==TRUE)
 events.infrequent <- t["Freq"]
 noevents <- diff(which(c(1,events.infrequent[,1])>=1))-1

setwd(output.directory)
filename.prefix <- paste(min(mot.data$Month_Yr), "-", max(mot.data$Month_Yr)," ")
filename <- paste(filename.prefix, i," g-chart.pdf")
pdf(filename, height = 7, width =12 )
t <- subset(t, t$Freq>=1)
par(mfrow=c(1,1))
qcc(noevents,
    type="g",
    #nsigmas=3,
    conf=0.95,
    labels=t[,"Var1"],
    axes.las=2,
    add.stats=FALSE,
    xlab= "Date",
    ylab = "Days between",
    title = paste(i, "\n", min(mot.data$Month_Yr), " to ", max(mot.data$Month_Yr)),
    restore.par = FALSE #to allow adding lines etc to chart with par command
)# Close qcc
dev.off()
} # Close For i


In [115]:
# code for calculating days between infrequent events
# pacu.data[which(pacu.data$Answer==events.descriptors[2]),"Day_Month_Yr"]

# CALCULATE ACHS INDICATORS and TABLE:
# ACHS indicators are:
# 1.1 Pre-anesthetic consultation by anaesthetist
# 1.2 Smoking cessation advice in pre-anaesthetic consultation
# 2.1 Presence of trained assistant
# 2.2 Documentation complies with ANZCA PS6
# 2.3 Stop-before you block procedure
# 2.4 Prophlactic antiemetics administered to patients wit a history
# 3.1 Relief of respiratory distress in recovery (re-intub/ LMA/ ventilation)
# 3.2 PONV treatment in PACU
# 3.3 Temp < 36C
# 3.4 Pain not responding to protocol
# 3.5 Unplanned stay > 2 hrs
# 4.1 Unplanned admission to ICU
# 4.2 Documented handover MOT-PACU
# 4.3 Documented handover PACU-Ward
# 5.1 Pain scores recorded for surgical patients
# 5.2 Post-op epidurals reviewed by anaesthetist daily
#

ACHS.1.1 <- percent(1)
ACHS.2.1 <- percent(length(mot.data[,"Tech.1.Name"])/length(mot.data[,"Anaes.1.Name"]))
ACHS.3.1 <- percent(sum(events.adverse$`Resp Serious`) / sum(monthlycases[,"PACU"]),format="d")
ACHS.3.2 <- percent(4/4038) # percent(sum(events.adverse$`PERSISTENT PONV`) / sum(monthlycases[,"PACU"]),format="d")
ACHS.3.3 <- percent(sum(events.adverse$`HYPOTHERMIA <36 DEG`) / sum(monthlycases[,"PACU"]),format="d")
ACHS.3.4 <- percent((sum(events.adverse$`ANAESTH R/V- PAIN`)+ sum(events.adverse$`PAIN R/V ANAES CONS`)) / sum(monthlycases[,"PACU"]),format="d")
ACHS.3.5 <- percent(sum(events.adverse$`PROLONGED STAY > 2 HRS`) / sum(monthlycases[,"PACU"]),format="d")

ACHS.table <- data.frame(
  Indicator = c("1.1 Peroperative consultation by anaesthetist",
                "2.1 Presence of trained assistant",
                "3.1 Relief of respiratory distress in recovery",
                "3.2 PONV treatment in PACU",
                "3.3 Temp < 36C",
                "3.4 Pain not responding to protocol",
                "3.5 Unplanned stay > 2 hrs"
    ),
  Numerator = c(length(mot.data[,"Anaes.1.Name"]),
                length(mot.data[,"Tech.1.Name"]), 
                sum(events.adverse$`Resp Serious`),
                4,
                sum(events.adverse$`HYPOTHERMIA <36 DEG`),
                sum(events.adverse$`ANAESTH R/V- PAIN`)+ sum(events.adverse$`PAIN R/V ANAES CONS`),
                sum(events.adverse$`PROLONGED STAY > 2 HRS`)),
  Denominator = c(length(mot.data[,"Anaes.1.Name"]),
                  length(mot.data[,"Anaes.1.Name"]),
                  sum(monthlycases[,"PACU"]),
                  sum(monthlycases[,"PACU"]),
                  sum(monthlycases[,"PACU"]),
                  sum(monthlycases[,"PACU"]),
                  sum(monthlycases[,"PACU"])),
  Value = c(ACHS.1.1, ACHS.2.1, ACHS.3.1, ACHS.3.2, ACHS.3.3, ACHS.3.4, ACHS.3.5)
) # Close ACHS.table <- df
# Write table to pdf
setwd(output.directory)
filename <- paste(filename.prefix, " ACHS Indicators.pdf")
#pdf(filename, height = 7, width =12 )
export_formattable(formattable(ACHS.table,align=c("l","r","r","r")), filename)
# dev.off()
formattable(ACHS.table,align=c("l","r","r","r"))

Indicator,Numerator,Denominator,Value
<fct>,<dbl>,<int>,<formttbl>
1.1 Peroperative consultation by anaesthetist,4800,4800,100.00%
2.1 Presence of trained assistant,4800,4800,100.00%
3.1 Relief of respiratory distress in recovery,6,4038,0.15%
3.2 PONV treatment in PACU,4,4038,0.10%
3.3 Temp < 36C,12,4038,0.30%
3.4 Pain not responding to protocol,24,4038,0.59%
3.5 Unplanned stay > 2 hrs,24,4038,0.59%
